In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data['headline']
    date = json_data['datePublished']
    date_m = json_data['dateModified']
    genre = json_data.get('genre', [])
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def get_link(start=0):
    notyet = []
    n_list = pd.read_json('nhkweb.json', encoding='utf-8')['url'].tolist()
    df_e = pd.read_json('nhkwebeasy.json', encoding='utf-8') 
    for i in df_e['url_normal'][start:]:
        if i not in n_list:
            notyet.append(i)
    return notyet

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic not in data:
            data.append(dic)
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [4]:
htmls = ['https://www3.nhk.or.jp/news/html/20191117/k10012180511000.html']

In [36]:
htmls = get_link(800); htmls

['https://www3.nhk.or.jp/news/html/20191010/k10012120321000.html',
 'https://www3.nhk.or.jp/news/html/20191017/k10012135411000.html']

# scrape

In [20]:
driver = webdriver.Chrome()

In [35]:
for nhkurl in tqdm.tqdm(htmls[:-2]):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'(between|1 time).*?<a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(2))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)





  0%|          | 0/51 [00:00<?, ?it/s]



  2%|▏         | 1/51 [01:30<1:15:38, 90.78s/it]



  4%|▍         | 2/51 [02:53<1:12:09, 88.36s/it]



  6%|▌         | 3/51 [04:01<1:05:47, 82.24s/it]



  8%|▊         | 4/51 [05:16<1:02:45, 80.11s/it]



 10%|▉         | 5/51 [06:28<59:37, 77.77s/it]



 12%|█▏        | 6/51 [07:45<58:09, 77.55s/it]



 14%|█▎        | 7/51 [08:54<54:54, 74.87s/it]



 16%|█▌        | 8/51 [10:10<53:48, 75.08s/it]



 18%|█▊        | 9/51 [11:18<51:14, 73.19s/it]



 20%|█▉        | 10/51 [12:28<49:19, 72.18s/it]



 22%|██▏       | 11/51 [13:42<48:25, 72.65s/it]



 24%|██▎       | 12/51 [14:52<46:43, 71.87s/it]



 25%|██▌       | 13/51 [15:52<43:19, 68.41s/it]



 27%|██▋       | 14/51 [16:53<40:47, 66.14s/it]



 29%|██▉       | 15/51 [17:53<38:34, 64.28s/it]



 31%|███▏      | 16/51 [19:08<39:24, 67.57s/it]



 33%|███▎      | 17/51 [20:30<40:42, 71.83s/it]



 35%|███▌      | 18/51 [22:02<42:52, 77.94s/it]



 37%|███▋      | 19/51 [23:35<43:58, 

In [21]:
html = driver.page_source.encode('utf-8')
dic = scrape(html)
js(dic)

In [15]:
re.findall(r'1 time(.............................)', html)

['</strong>&nbsp;<a href="/web/']